In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd /content/drive/MyDrive/ytvideo-comment-recording


In [ ]:
# 出力先のデータベース (SQLite3)
save_db_path = "out.db"

# 対象のYouTube動画のURL
videos = [
    "https://www.youtube.com/watch?v=ieMKBctJ2aM"
]


In [ ]:
%pip install yt-dlp==2023.11.16
%pip install openai-whisper==20231117


In [ ]:
import whisper
import torch
from yt_dlp import YoutubeDL
import sqlite3
import os
import tempfile
from typing import Optional


In [ ]:
# GPUが使える場合はGPUを使う
if not torch.cuda.is_available():
    print("Warning: CUDA is not available. The model will run on CPU.")


In [ ]:
print(">> Loading speech to text model...")
# 大規模モデルを読み込む
whisper_model = whisper.load_model("large")
print("<< Loaded speech to text model\n")


In [ ]:
class Database:
    def __init__(self, db_path: str):
        self.conn = sqlite3.connect(db_path)
        self.__create_tables()

    def insert_yt_video_no_commit(
        self,
        yt_id: str,
        title: str
    ):
        cursor = self.conn.cursor()
        cursor.execute(
            "INSERT INTO yt_videos (yt_id, title) VALUES (?, ?)",
            (yt_id, title)
        )

    def insert_talk_no_commit(
        self,
        yt_id: str,
        start_sec: int,
        end_sec: int,
        comment: str
    ):
        cursor = self.conn.cursor()
        cursor.execute(
            "INSERT INTO talks (yt_id, start_sec, end_sec, comment) VALUES (?, ?, ?, ?)",
            (yt_id, start_sec, end_sec, comment)
        )

    def commit(self):
        self.conn.commit()

    def close(self):
        self.conn.close()

    def __create_tables(self):
        cursor = self.conn.cursor()

        # yt_videos テーブルが存在しなければ作成
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS yt_videos (
                yt_id VARCHAR(32) NOT NULL PRIMARY KEY,
                title VARCHAR(255) NOT NULL
            )
        """)

        # talks テーブルが存在しなければ作成
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS talks (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                yt_id VARCHAR(32) NOT NULL,
                start_sec INTEGER NOT NULL,
                end_sec INTEGER NOT NULL,
                comment TEXT NOT NULL,
                FOREIGN KEY (yt_id) REFERENCES yt_videos(yt_id)
            )
        """)

        self.conn.commit()


In [ ]:
def download_video_from_youtube(
    video_url: str,
    save_dir: str
) -> Optional[tuple[str, str, str]]:

    options = {
        "format": "bestaudio/best",
        "outtmpl": os.path.join(save_dir, "%(id)s.%(ext)s")
    }

    with YoutubeDL(options) as ydl:
        info_dict = ydl.extract_info(video_url, download=False)
        video_id = info_dict.get("id", None)
        video_ext = info_dict.get("ext", None)
        video_title = info_dict.get("title", None)

        if not video_id:
            return None

        ydl.download([video_url])
        return (
            os.path.join(save_dir, f"{video_id}.{video_ext}"),
            video_id,
            video_title
        )


In [ ]:
# データベースインスタンスを作り、データベースに接続する
db = Database(save_db_path)


In [ ]:
for video in videos:
    with tempfile.TemporaryDirectory() as temp_dir:
        print(f">> Downloading video: {video}")

        # YouTubeから動画をダウンロードする
        video_path, video_id, video_title = download_video_from_youtube(video, temp_dir)
        if not video_path:
            continue

        print(f"   Downloaded video: {video_title} ({video_path})")

        # 動画から文字起こしをする
        result = whisper_model.transcribe(
            video_path,
            verbose=False,
            fp16=False,
            language="ja"
        )

        # 動画情報を格納
        db.insert_yt_video_no_commit(
            video_id,
            video_title
        )

        # 文字起こし結果を格納
        for segment in result["segments"]:
            db.insert_talk_no_commit(
                video_id,
                int(segment["start"]),
                int(segment["end"]),
                segment["text"]
            )

        # データベースに反映
        db.commit()

        print("<< Transcribed video\n")


In [ ]:
# データベースと切断する
db.close()
